In [2]:
import os
import boto3
import json
import pandas

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"

index_name =  os.environ.get('AWS_OPENSEARCH_INDEX_NAME')
service = 'aoss'
host = os.getenv('AWS_OPENSEARCH_ENDPOINT')
region = os.getenv('AWS_REGION')


df = pandas.read_parquet('../data/bger-2024-3-text.parquet')


# Notes for porting to AWS Serverless

Recommendation: Start with AWS Lambda
For your use case (encoding a single string with distiluse-base-multilingual-cased-v2), AWS Lambda is the better choice because:
It’s more cost-effective for low-to-moderate traffic.

The model size (135M parameters) and task (encoding a single string) fit within Lambda’s memory and timeout constraints.

Using a container image and ONNX optimization simplifies deployment and reduces latency.

Steps to Get Started:
Convert the model to ONNX to reduce size and inference time.

Build a Docker image with the model and sentence-transformers.

Deploy to Lambda with 1024-2048 MB memory and a 60-second timeout.

Test with a sample invocation and monitor cold start performance.

If latency is an issue, consider SageMaker Serverless or Provisioned Concurrency.

If you encounter issues (e.g., package size or library conflicts), refer to the Stack Overflow discussion on Lambda deployment errors and ensure a CPU-only PyTorch build.



In [4]:
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

sts = session.client("sts")
identity = sts.get_caller_identity()
print("Running as IAM identity:", identity["Arn"])

aoss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_compress=True,
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
    timeout=36000,  # Set request timeout in seconds
)

# Check if index exists
try:
    exists = aoss_client.indices.exists(index=index_name)
    print(f"Index {index_name} exists: {exists}")
    if not exists:
        raise {"statusCode": 500, "body": json.dumps(f"Index {index_name} does not exist")}
except Exception as e:
    print(f"Error checking index: {str(e)}")
    raise {"statusCode": 500, "body": json.dumps(f"Error checking index: {str(e)}")}

Running as IAM identity: arn:aws:iam::211125557955:user/Christophe
Index fed-court-chunks-index exists: True


In [5]:

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  },
  {
    "title": "Agissements potentiellement illégaux de la société TVT",
    "content": "Un avis, intitulé « Avis de travaux », de la société TVT a été affiché à l'entrée d'un immeuble situé dans la commune d'Ecublens, canton de Vaud. Un résident de cet immeuble s'inquiète de la véracité de cet avis, relevant que ni les locataires, ni la gérance, ni les propriétaires n'ont commandé de travaux. L'avis prétend également que « L'installation est obligatoire et gratuite », une affirmation pour laquelle le résident ne perçoit aucune base légale. Au-delà d'une simple pratique commerciale agressive, le résident craint que la société TVT ne se prévale de l'autorité communale pour s'introduire par la ruse dans les propriétés privées et se demande si de telles actions sont punies pénalement en vertu du droit suisse."
  }
]


In [7]:
from pathlib import Path
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer
onnx_path = Path("../data/onnx_model").resolve()
tokenizer = AutoTokenizer.from_pretrained(str(onnx_path))

# Load ONNX model directly
session = ort.InferenceSession(str(onnx_path / "model.onnx"))

# Prepare inputs
case_number = 10
sentence = test_cases[case_number]["content"]
inputs = tokenizer(sentence, return_tensors="np")

# Rename input keys if needed (depends on your export)
onnx_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}

# Run inference
outputs = session.run(None, onnx_inputs)[0]
sentence_embedding = outputs[0]
# Use embedding for search
top_matches = 10
response = aoss_client.search(
    index=index_name,
    body={
        "size": top_matches,
        "query": {
            "knn": {
                "embedding": {
                    "vector": sentence_embedding.tolist(),
                    "k": top_matches
                }
            }
        }
    },
    timeout=30
)

hit_ids = set()
for hit in sorted(response["hits"]["hits"], key=lambda x: x["_source"]["doc_id"], reverse=True):
    hit_ids.add(hit["_source"]["doc_id"])
    print(hit["_source"]["doc_id"], hit["_score"])

relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

5D_35/2020 0.6954806
5D_182/2017 0.7024482
5A_535/2010 0.6955875
4A_238/2023 0.70010597
2C_812/2013 0.7003414
2C_465/2020 0.70129627
1F_18/2024 0.6987176
1F_17/2024 0.6987176
1C_624/2013 0.70453346
1C_534/2012 0.6950613


[{'doc': 'Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal {T 0/2} 5A_535/2010 Urteil vom 10. August 2010 II. zivilrechtliche Abteilung Besetzung Bundesrichterin Escher, präsidierendes Mitglied, Bundesrichter L. Meyer, von Werdt, Gerichtsschreiber Möckli. Verfahrensbeteiligte X.________, vertreten durch Rechtsanwalt Eric Stern, Beschwerdeführerin, gegen Y.________, vertreten durch Rechtsanwalt Bernhard Tröhler, Beschwerdegegner. Gegenstand Kinderrückführung, Beschwerde gegen den Beschluss des Obergerichts des Kantons Zürich, II. Zivilkammer, vom 13. Juli 2010. Sachverhalt: A. Y.________ und X.________ sind die Eltern der am 27. Oktober 2007 in A.________ geborenen Zwillinge B.________ und C.________. Der Vater wurde ab Februar 2007 in D.________ nach Aufwand besteuert, da er eine Wohnung an der E.________strasse xxx gemietet und "steuerrechtlichen Wohnsitz" in der Schweiz genommen hatte. Unbestrittenermassen lebten die Parteien bis Mitte 2008 gemeinsam mit den Kindern

In [8]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Décision {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des décisions fédérales suisses fournies, voici un avis juridique concernant la situation décrite :

1. **Caractère obligatoire et gratuité de l'installation annoncée par la société TVT**  
L'avis affiché par la société TVT prétend que « L'installation est obligatoire et gratuite ». Or, en droit suisse, une telle affirmation doit reposer sur une base légale claire et explicite. Aucune des décisions fournies ne mentionne directement ce type d'installation, mais le principe général est que toute intervention dans une propriété privée doit être fondée sur un droit légal ou contractuel.  
Par exemple, dans la décision 5D_35/2020 relative à des rapports de voisinage, le Tribunal fédéral souligne l'importance de la détermination précise des droits et obligations, notamment en matière d'accès et d'intervention sur un fonds privé. Une installation prétendument obligatoire sans base légale pourrait être contestée.  
De plus, dans la décision 1C_624/2013, le Tribunal fédéral rappelle

In [3]:
df

,docref,url,date,year,proc_type,merged_cases,division,division_type,n_judges,language,...,resp_represented,outcome,outcome_binary,cited_bger,n_cited_bger,cited_bge,n_cited_bge,leading_case,text,doi_version
0,1B_8/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Criminal,False,1. Public,Public,3.0,de,...,False,rejected,False,1B_8/2007,1,129 I 361;127 II 32;129 I 361;123 I 268,3,None,Tribunale federale Tribunal federal {T 0/2} 1B...,10.5281/zenodo.14867950
1,1B_19/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Criminal,False,1. Public,Public,3.0,fr,...,False,rejected,False,1B_19/2007;1S.1/2006,2,123 I 268;124 I 336;116 Ia 143;123 I 31;116 Ia...,8,None,Tribunale federale Tribunal federal {T 0/2} 1B...,10.5281/zenodo.14867950
2,1C_16/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-28,2007.0,Public,True,1. Public,Public,1.0,de,...,False,inadmissible,False,1C_16/2007;1C_18/2007,2,,0,None,Tribunale federale Tribunal federal {T 0/2} 1C...,10.5281/zenodo.14867950
3,5A_39/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-27,2007.0,Civil,False,2. Civil,Civil,1.0,de,...,False,inadmissible,False,5A_39/2007,1,109 II 350;118 II 254,2,None,Tribunale federale Tribunal federal {T 0/2} 5A...,10.5281/zenodo.14867950
4,2C_29/2007,https://www.bger.ch/ext/eurospider/live/it/php...,2007-02-27,2007.0,Public,False,2. Public,Public,3.0,de,...,False,rejected,False,2C_29/2007;2C_1/2007,2,131 II 449,1,None,Tribunale federale Tribunal federal {T 0/2} 2C...,10.5281/zenodo.14867950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127472,1C_642/2023,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-30,2024.0,Public,False,1. Public,Public,3.0,de,...,False,rejected,False,1C_642/2023,1,149 II 337;148 II 392;148 V 209;149 II 337;148...,4,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127473,7B_679/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-19,2024.0,Criminal,False,2. Criminal,Criminal,1.0,de,...,True,writeoff,None,7B_679/2024;7B_322/2023;7B_323/2023;7B_324/2023,4,,0,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127474,4A_423/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-17,2024.0,Civil,False,1. Civil,Civil,3.0,fr,...,True,rejected,False,4A_423/2024;4A_458/2022;4A_509/2021;4A_509/202...,9,145 I 239;138 III 46;140 III 86;137 III 580;13...,29,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
127475,4A_284/2024,https://www.bger.ch/ext/eurospider/live/it/php...,2024-12-17,2024.0,Civil,False,1. Civil,Civil,3.0,fr,...,True,granted,True,4A_284/2024;4A_468/2020;4A_347/2016;4A_468/202...,14,140 III 115;137 I 58;137 II 353;137 III 226;13...,20,None,Bundesgericht Tribunal fédéral Tribunale feder...,10.5281/zenodo.14867950
